In [1]:
import pandas as pd
import numpy as np
import folium
import requests
import json
import pprint
import datetime as dt
import re
import sympy
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import csv

In [2]:
room_data = pd.read_csv('room.csv',encoding='CP949')
room_data.head()

,건물명(상호명),건물위치,주택유형구분,세대수(객실수),건축연도
0,NaN,전라북도 전주시 덕진구 고랑동 819-5,다세대주택(3세대),3,19920810
1,NaN,전라북도 전주시 덕진구 금암동 1569-6,다세대주택(3세대),6,19891206
2,NaN,전라북도 전주시 덕진구 금암동 1588-23,다세대주택(3세대),3,19890803
3,휴엔하임 주상복합 아파트,전라북도 전주시 덕진구 금암동 1639-1,"공동주택(아파트), 업무시설(오피스텔), 근린생활시설",30,20161128
4,NaN,전라북도 전주시 덕진구 금암동 525-83,다세대주택,4,19891121


In [3]:
room_data['세대수(객실수)'].value_counts()

3      1332
10     1066
11      788
9       643
4       616
5       576
8       542
2       497
6       423
12      349
7       302
15      296
13      246
18      201
1       192
0       169
14      134
16      129
19      111
17       91
30        4
20        3
54        2
159       2
56        2
24        1
48        1
288       1
182       1
110       1
73        1
172       1
44        1
34        1
826       1
77        1
53        1
27        1
51        1
167       1
276       1
28        1
191       1
Name: 세대수(객실수), dtype: int64

In [4]:
room_data['주택유형구분'].value_counts()

단독주택                             3751
단독주택,제1종근린생활시설                    679
다가구주택                             494
단독주택(다가구주택)                       417
단독주택(3가구)                         196
                                 ... 
단독주택(다가구), 제2종근린생활시설                1
단독주택(3가구), 제1종근린생활시설                1
단독주택(다가구주택,9가구)                     1
다가구주택(10가구) 및 제1종 근린생활시설(소매점)       1
단독주택(18)                            1
Name: 주택유형구분, Length: 741, dtype: int64

In [5]:
room_data['주택유형구분'].unique()

array(['다세대주택(3세대)', '공동주택(아파트), 업무시설(오피스텔), 근린생활시설', '다세대주택', '공동주택',
       '아파트,업무시설,근린생활시설', '다세대주택(2세대)', '다세대주택(4세대), 창고, 화장실', '다가구주택',
       '다세대주택(4세대)', '다세대주택(5세대)', '다세대주택, 근린생활시설(사무실)', '주택, 소매점',
       '근린생활시설, 주택', '다세대주택, 창고', '다세대주택, 변소, 창고', '차고, 다세대주택',
       '근린생활시설, 단독주택(4가구)', '근린생활시설, 단독주택(5가구)',
       '공동주택(도시형생활주택(원룸형)),업무시설', '다세대주택(5가구)', '제1,2종근린생활시설,업무시설(오피스텔)',
       '단독주택', '단독주택(다가구주택)', '다세대주택, 근린생활시설', '단독주택(다가구)', '단독주택(3가구용)',
       '단독주택(5가구)', '단독주택(4가구)', '단독주택(다가구주택), 근린생활시설',
       '단독주택, 제1,2종근린생활시설', '단독주택(다가구주택), 제2종근린생활시설', '다가구주택, 제2종근린생활시설',
       '단독주택(다가구13가구)', '근린생활시설, 단독주택', '다세대주택, 변소', '단독주택,다중주택',
       '단독주택, 근린생활시설', '단독주택(19)', '단독주택(18)', '단독주택(다가구주택,9가구)',
       '단독주택 및 제1종근린생활시설', '단독주택(19가구)', '단독주택(17가구)', '단독주택(7가구)',
       '주택, 변소, 창고', '단독주택, 제2종근린생활시설', '주택', '단독주택(다가구주택(3가구))',
       '단독주택(2가구)', '다세대주택, 화장실', '주택, 근린생활시설(소매점)', '주택(2가구)',
       '제1종근생(소매점) 및 단독주택(3가구)', '다세대 주택, 화장실', '주택(5가구)',
       '단

In [6]:
room_data.describe()

,세대수(객실수)
count,8734.000000
mean,8.135791
std,11.724445
min,0.000000
25%,3.000000
50%,8.000000
75%,11.000000
max,826.000000


In [7]:
p = re.compile(r'\d+')
room_data['건축연도'] = room_data['건축연도'].map(lambda x: x if p.search(x) else '0')
room_data.drop(room_data[room_data['건축연도'].map(lambda x:int(x[:4])) < 1945].index, inplace=True) #해방 전 옛 건물들 제거
room_data.drop(room_data[room_data['세대수(객실수)']<1].index, inplace=True) #세대수가 0인 것 제거
room_data['const_year_point'] = 5-(dt.datetime.today().year-room_data['건축연도'].map(lambda x: int(x[:4])))//5

room_data.describe()

,세대수(객실수),const_year_point
count,8555.000000,8555.000000
mean,8.301344,2.547165
std,11.789038,1.858123
min,1.000000,-10.000000
25%,4.000000,1.000000
50%,8.000000,3.000000
75%,11.000000,4.000000
max,826.000000,5.000000


In [8]:
class Room:
    def __init__(self, name):
        self.name = name
    
    def location(self,x,y):
        self.x = x
        self.y = y
    
    def bus(self, bus_route, bus_amount):
        self.bus_route = bus_route
        self.bus_amoount = bus_amount
    
    def safety(self, cctv, police):
        self.cctv = cctv
        self.police = police
        
    def natural(self, park, river):
        self.park = park
        self.river = river
        
    def price(self, contract_type, deposit,monthly_fee):
        self.contract_type = contract_type
        self.deposit = deposit
        self.monthly_fee = monthly_fee

In [9]:
# 건물 위치를 참조하여 folium에 좌표 찍기
# 건축 연도를 참조하여 room_data에 신축 / 준신축 / 보통 / 재건축필요 등의 등급으로 분류
## 6134개의 건물 데이터

# room_data['건물위치']
# key = "7F8A26C5-7C0A-318B-A521-FE1AA35942DB"
# arr = []
# f = open("./loc_all.txt", 'w')
# f.close()
# import time
# for i in room_data['건물위치']:
#     f = open("./loc_all.txt", 'a')
#     location = i.strip()
#     response = requests.get(f"http://api.vworld.kr/req/address?service=address&request=getCoord&key={key}&address={location}&type=PARCEL")
#     if 'result' in response.json()['response']:
#         x= float(response.json()['response']['result']['point']['x'])
#         y = float(response.json()['response']['result']['point']['y'])
#         arr.append([float(x),float(y)])
#         f.write(str(x))
#         f.write(',')
#         f.write(str(y))
#         f.write(',')
#         f.write(location)
#         f.write('\n')
#         f.close()
#     else:
#         print(location, response.json())
#     time.sleep(0.1)

In [10]:
f = open('./loc_all.txt', 'r')
lines = f.readlines()

In [11]:
# bus_data = open('./bus_df.csv', 'r')
# bus_rine = bus_data.readlines()
# bus_dict = dict()
    
map1 = folium.Map(location=[35.83001,127.125001], zoom_start=13)
arr = []
arr_dict = dict()
for line in lines:
    lat, long, addr = line.split(',')
    addr = list(addr.split())
    lat = round(float(lat),2)
    long = round(float(long),2)
    if [lat,long] not in arr:
        arr.append([lat,long])
        arr_dict[f'{lat}-{long}'] = dict()
        arr_dict[f'{lat}-{long}']['value'] = 1
        arr_dict[f'{lat}-{long}']['addr'] = [' '.join(addr[:4])]
    else:
        arr_dict[f'{lat}-{long}']['value'] +=1
        arr_dict[f'{lat}-{long}']['addr'].append(' '.join(addr[:4]))

# DRAWING MAP
# for content in arr_dict:
#     lat,long = map(float, content.split('-'))
#     house_amount = arr_dict[f'{lat}-{long}']['value']
#     addr = max(arr_dict[f'{lat}-{long}']['addr'])
#     iframe = folium.IFrame(f'''
#             {addr} 지역의<br>
#             주택 갯수 = {house_amount}<br>
#             좌표 = {lat}-{long}
#                 ''')
    
#     area100_iframe = folium.IFrame(f'''
#             {addr}의 원룸 반경 100m
#                 ''')
    
#     area500_iframe = folium.IFrame(f'''
#             {addr}의 원룸 반경 500m
#                 ''')
    
#     popup = folium.Popup(iframe,
#              min_width=400,
#              max_width=200)
    
#     area100_popup = folium.Popup(area100_iframe,
#              min_width=400,
#              max_width=200)
    
#     area500_popup = folium.Popup(area500_iframe,
#              min_width=400,
#              max_width=200)    

#     folium.Marker([long, lat], popup=popup, icon=folium.Icon(color='red',icon='info-sign')).add_to(map1)
#     folium.Circle([long, lat], radius=100, color='#3186cc', fill_color='#3186cc', popup=area100_popup).add_to(map1)
#     folium.Circle([long, lat], radius=500, color='#ebe134', fill_color='#ebe134', popup=area500_popup).add_to(map1)
# map1.save('map1.html')

## Transport

In [12]:
# url = "http://openapi.jeonju.go.kr/jeonjubus/openApi/traffic/bus_location2_stopnm_common.do?serviceKey=cjE984BWrGFH3DFA2kLDoNx7j950nWjp6Ajz0CBaEABY5arYHY1C80RNBw6ptmayEV6B0zzpOfAbB73miZ4gnA%3D%3D&searchFld=searchNM&searchNm=%EC%95%88%EA%B3%A8"
# response = requests.get(url)
# data = BeautifulSoup(response.text, 'xml')
# stopX = data('stopX')
# stopY = data('stopY')
# stopKname = data('stopKname')
# stopId = data('stopStandardid')
# bus_stop_arr = []

# for i in range(len(stopX)):
#     x=float(stopX[i].text)
#     y=float(stopY[i].text)
#     name = stopKname[i].text
#     stop_id = int(str(stopId[i].text).strip())
#     if 35.78<y and y<35.94 and 127.06<x and x<127.19:
#         bus_stop_arr.append([x,y,name,stop_id])
# bus_stop_arr.sort(key=lambda x:x[0])

In [13]:
# bus_stop_arr_df = pd.DataFrame(arr)
# bus_stop_arr_df

## Bus

주택가 기준으로 근처 500m 이내의 버스정류장 갯수와 그 정류장을 지나는 버스 노선의 수를 정리함.

In [14]:
# bus_df = pd.DataFrame(arr)
# bus_df['near_bus_stop']=''
# bus_df['bus_at_each_stop']=''
# bus_df['total_bus']=''
# i=0
# for content in arr[5:]: 
#     station_count = 0
#     bus_count = 0
#     content_lat = content[0]
#     content_long = content[1]
#     near_bus_stop = []
#     bus_route_arr = []
#     for bus_stop in bus_stop_arr:
#         if bus_stop[0]>content_lat-0.005 and bus_stop[0]<content_lat+0.005 and bus_stop[1]>content_long-0.005 and bus_stop[1]<content_long+0.005:
#             near_bus_stop.append(bus_stop)
#     bus_df['near_bus_stop'][i]=near_bus_stop

    
#     for bus_stop in near_bus_stop:
#         if ((content_lat-bus_stop[0])**2+(content_long-bus_stop[1])**2)**(1/2) <= ((content_lat-(content_lat-0.0055))**2+(content_long-(content_long-0.0044))**2)**(1/2):
#             bus_stop_id = bus_stop[3]
#             url = f'http://openapi.jeonju.go.kr/jeonjubus/openApi/traffic/general_busstop_common.do?serviceKey=cjE984BWrGFH3DFA2kLDoNx7j950nWjp6Ajz0CBaEABY5arYHY1C80RNBw6ptmayEV6B0zzpOfAbB73miZ4gnA%3D%3D&stopStandardid={bus_stop_id}'
#             response = requests.get(url)
#             data = BeautifulSoup(response.text, 'xml')
#             all_route=data('brtStdId')
#             this_bus_route_count=0
#             for route in range(len(all_route)):
#                 route_id = all_route[route]
#                 if route_id not in bus_route_arr:
#                     bus_route_arr.append(route_id)
#                     this_bus_route_count+=1
#             if this_bus_route_count>=1:
#                 bus_df['bus_at_each_stop'][i]+=f'({bus_stop[2]}-{bus_stop[3]})-{this_bus_route_count}대'
#         bus_df['total_bus'][i]=len(bus_route_arr)
#     i+=1
#     bus_df.to_csv('bus_df.csv', sep=',',na_rep='NaN', encoding='cp949', index=False)

In [15]:
# # CCTV
cctv_url =  'http://openapi.jeonju.go.kr/rest/cctv/getCctvList?serviceKey=cjE984BWrGFH3DFA2kLDoNx7j950nWjp6Ajz0CBaEABY5arYHY1C80RNBw6ptmayEV6B0zzpOfAbB73miZ4gnA%3D%3D&pageSize=1400&startPage=1'
cctv_response = requests.get(cctv_url)
cctv_data = BeautifulSoup(cctv_response.text, 'xml')
cctv_posX = cctv_data('cctvPosx')
cctv_posY = cctv_data('cctvPosy')
dong_addr = cctv_data('dongAddr')
cctv_arr = []
cctv_dict = dict()
for i in range(len(cctv_posX)):
    if len(cctv_posX[i].text) > 3 and len(cctv_posY[i].text)>3:
        x_position = round(float(cctv_posX[i].text),3)
        y_position = round(float(cctv_posY[i].text),3)
        if [x_position,y_position] not in cctv_arr:
            cctv_arr.append([x_position,y_position])
           
        
# cctv_map = folium.Map(location=[35.83001,127.125001], zoom_start=13)
            
# Police_box
# dukjin_jigudae_addr = ['전주시 덕진구 안덕원로 173', '전주시 덕진구 진버들4길 10', '전주시 덕진구 권삼득로 396']
# dukjin_pachulso_addr = ['전주시 덕진구 진북1길 19', '전주시 덕진구 동부대로 871', '전주시 덕진구 쪽구름로 82', '전주시 덕진구 신복로 58', 
#                        '전주시 덕진구 솔내6길 23', '전주시 덕진구 송천로 50', '전주시 덕진구 동부대로 688', '전주시 덕진구 출판로 92']
# dukjin_police_addr = ['전북 전주시 덕진구 온고을로 299']
# wansan_jigudae_addr = ['전주시 완산구 대동로 95', '전주시 완산구 성지산로 24', '전주시 완산구 서원로 295', '전주시 완산구 새터로46', 
#                        '전주시 완산구 용머리로 125', '전주시 완산구 마전들로 1', '전주시 완산구 평화로 159']
# wansan_pachulso_addr = ['전주시 완산구 서학 3길 46']
# wansan_police_addr = ['전주시 완산구 전라감영로 66']      
# key = "7F8A26C5-7C0A-318B-A521-FE1AA35942DB"
# police_addr = [dukjin_jigudae_addr, dukjin_pachulso_addr, dukjin_police_addr, wansan_jigudae_addr, wansan_pachulso_addr, wansan_police_addr]
# police_loc = []
# for sub_addr in police_addr:
#     for location in sub_addr:
#         response = requests.get(f"http://api.vworld.kr/req/address?service=address&request=getCoord&key={key}&address={location}&type=ROAD")
#         if 'result' in response.json()['response']:
#             x= float(response.json()['response']['result']['point']['x'])
#             y = float(response.json()['response']['result']['point']['y'])
#             police_loc.append([x,y])

police_loc = [[127.146461566, 35.834468712], [127.16380635, 35.835774879], [127.120456415, 35.848296399], 
              [127.132636253, 35.826902578], [127.149387537, 35.863971253], [127.076131218, 35.872743759],
              [127.107977884, 35.856099777], [127.123538425, 35.858744294], [127.118451299, 35.857844477], 
              [127.160431827, 35.850066374], [127.062692384, 35.839356441], [127.092696544, 35.84296123], 
              [127.146036617, 35.823575242], [127.121085454, 35.79975135], [127.126785309, 35.812004112], 
              [127.111398444, 35.835411819], [127.123993749, 35.806270762], [127.096152619, 35.821300798], 
              [127.135674461, 35.791003554], [127.151890494, 35.807513991], [127.146370384, 35.815267116]]

### 편의점
conv_df = pd.read_csv('conv_store.csv', encoding='cp949')
conv_df['위도'].dropna()
conv_jeonju = conv_df[conv_df['주소'].str.contains('전북 전주')]
conv_jeonju.head()
x_position = list(x for x in conv_jeonju['위도'])
y_position = list(y for y in conv_jeonju['경도'])
name = list(z for z in conv_jeonju['편의점지점명'])
pol_dict = dict()
conv_dict = dict()
for content in arr:
    cctv_dict.setdefault(f'{content[0]}-{content[1]}', 0)
    pol_dict.setdefault(f'{content[0]}-{content[1]}', 0)
    conv_dict.setdefault(f'{content[0]}-{content[1]}', 0)

for content in arr:
    cctv_count = 0
    police_count = 0
    conv_count = 0
    content_lat = content[0]
    content_long = content[1]
    lat,long = content
    
    for cctv in cctv_arr:
        if ((content_lat-cctv[0])**2+(content_long-cctv[1])**2)**(1/2) <= ((content_lat-(content_lat-0.0055))**2+(content_long-(content_long-0.0055))**2)**(1/2):
            cctv_dict[f'{lat}-{long}']+=1
    for police in police_loc:
        if ((content_lat-police[0])**2+(content_long-police[1])**2)**(1/2) <= ((content_lat-(content_lat-0.0055))**2+(content_long-(content_long-0.0055))**2)**(1/2):
            pol_dict[f'{lat}-{long}']+=1
    for i in range(len(x_position)):
        if ((content_lat-y_position[i])**2+(content_long-x_position[i])**2)**(1/2) <= ((content_lat-(content_lat-0.0055))**2+(content_long-(content_long-0.0055))**2)**(1/2):
            conv_dict[f'{lat}-{long}']+=1
    
    
#     iframe = folium.IFrame(f'''{long}-{lat}의 원룸 반경 500m: 
#                             <br> CCTV 갯수 : {cctv_count}
#                             <br> 파출소 갯수 : {police_count}
#                             <br> 버스노선 수 :
#                             <br> 근처 버스정류장 수 :
#                             <br> 근처 편의점 수 : {conv_count}
#                             ''')
#     popup = folium.Popup(iframe,
#                      min_width=500,
#                      max_width=500)
    
#     folium.Marker([long, lat], popup=f'{long}-{lat}의 원룸', icon=folium.Icon(color='red',icon='info-sign')).add_to(cctv_map)
#     folium.Circle([long, lat], radius=100, color='#3186cc', fill_color='#3186cc', popup=f'{long}-{lat}의 원룸 반경 100M').add_to(cctv_map)
#     folium.Circle([long, lat], radius=500, color='#ebe134', fill_color='#ebe134', popup=popup).add_to(cctv_map)
            
# for cctv in cctv_arr:
#     folium.Marker([cctv[1], cctv[0]], popup=f'위도 : {cctv[1]}-경도 : {cctv[0]}', icon=folium.Icon(color='black',icon='warning-sign')).add_to(cctv_map)
# cctv_map.save('map_cctv.html')

In [16]:
### 공원
# park_addr = dict()
# park_addr_road = {
#     '덕진공원':'전북 전주시 덕진구 권삼득로 390 전주덕진공원',
#     '완산공원':'전북 전주시 완산구 공수내1길 19-4',
#              '중산공원':'전북 전주시 완산구 메너머4길 11',
#              '거마공원':'전북 전주시 완산구 성지산로 6',
#              '혼불문학공원':'전북 전주시 덕진구 연화길 19',
#              '서일공원':'전북 전주시 완산구 여울로 77',
#              '그린공원':'전북 전주시 완산구 거마남로 41',
#              '용호근린공원':'전북 전주시 완산구 홍산로 148',
#              '효자공원':'전북 전주시 완산구 서원로 238',
#              '이바디공원':'전북 전주시 완산구 우전로 177',
#              '인정공원':'전북 전주시 완산구 인정2길 5',
#                  '문학공원':'전북 전주시 완산구 유연로 217-24',
#              '팔복도시공원':'전북 전주시 덕진구 서귀로 107 전주시청소년자유센터',
#              '만남의광장':'전북 전주시 덕진구 완주로 6',
#              '솔내생활체육공원':'전북 전주시 덕진구 고내천변로 58',
#              '솔바람공원':'전북 전주시 완산구 우전로 130',
#              '강변공원':'전북 전주시 완산구 강변로 96-14',
#              '풍남공원':'전북 전주시 완산구 효자천변1길 31',
#              '효문공원':'전북 전주시 완산구 거마평로 109-16',
#              '양지공원':'전북 전주시 완산구 송정로 15',
#              '송정공원':'전북 전주시 완산구 하거마7길 43',
#              '노송공원':'전북 전주시 완산구 문화3길 14-3',
# }
# park_addr_parcel = {
#                 '다가공원':'전북 전주시 완산구 중화산동1가 150-3',
#              '가련산공원':'전북 전주시 덕진구 덕진동2가 산27-1',
#              '중상보도시근린공원':'전북 전주시 덕진구 우아동2가 907',
#              '덕진체련공원':'전북 전주시 덕진구 덕진동1가 640-10',
#              '아중체련공원':'전북 전주시 덕진구 우아동1가 1020',
#              '평화생태공원':'전북 전주시 완산구 평화동2가 889-3',
#              '화정공원':'전북 전주시 덕진구 송천동2가 1356',
#               '하늘공원':'전북 전주시 덕진구 덕진동2가 682',
#              '전주천생태학습장':'전북 전주시 덕진구 덕진동2가 646-1',
#              '태평문화공원':'전북 전주시 완산구 태평동 293',
#              '삼천체육소공원':'전북 전주시 완산구 삼천동2가 730-15',
#                  '백로공원':'전북 전주시 완산구 효자동2가 1371',
#              '바위백이공원':'전북 전주시 완산구 효자동3가 1858',
#              '마전숲공원':'전북 전주시 완산구 효자동3가 1526-1',
#              '찬새암공원':'전북 전주시 완산구 평화동2가 858-4',
#                  '메너머공원':'전북 전주시 완산구 중화산동2가 702-1',
#              '고사평서공원':'전북 전주시 완산구 서신동 944-1',
#              '서곡공원':'전북 전주시 덕진구 팔복동3가 46-1',
#              '혁신공원':'전북 전주시 덕진구 만성동 1353-9',
#              '엽순근린공원':'전북 전주시 덕진구 장동',
#              '오공근린공원':'전북 전주시 덕진구 중동 830',
#                  '오동제공원':'전북 전주시 덕진구 우아동2가 910-4',
#              '명당골공원':'전북 전주시 완산구 서신동 887-8',
#              '효자제1호공원':'전북 전주시 완산구 효자동3가 1604-29',
#              '효자제2호공원':'전북 전주시 완산구 효자동3가 1615-22',
#              '효자제3호공원':'전북 전주시 완산구 효자동3가 1641-7',
#              '효자제4호공원':'전북 전주시 완산구 효자동3가 1687-13',
#              '효자제5호공원':'전북 전주시 완산구 효자동3가 1713-18',
#              '효자제6호공원':'전북 전주시 완산구 효자동2가 1247-13',
#              '효자제7호공원':'전북 전주시 완산구 효자동2가 1260-18',
#              '효자제8호공원':'전북 전주시 완산구 효자동2가 1164-1',
#              '맏내공원':'전북 전주시 완산구 평화동2가 873-5',
#              '신성공원':'전북 전주시 완산구 평화동2가 230-133',
#              '평화2공원':'전북 전주시 완산구 평화동1가 722-1',
#              '서학광장':'전북 전주시 완산구 서서학동 296-1',
#              '장승백이소공원':'전북 전주시 완산구 서서학동 산27-2',
#             '푸른연꽃공원':'전북 전주시 완산구 효자동2가 1205-1',
#                  '싸이언스공원':'전북 전주시 완산구 효자동2가 1310',
#              '한지공원':'전북 전주시 완산구 효자동2가 1191-3',
# }

# for j in park_addr_road.values():
#     key = "7F8A26C5-7C0A-318B-A521-FE1AA35942DB"
#     location = j
#     response = requests.get(f"http://api.vworld.kr/req/address?service=address&request=getCoord&key={key}&address={location}&type=ROAD")
#     if 'result' in response.json()['response']:
#         x= float(response.json()['response']['result']['point']['x'])
#         y = float(response.json()['response']['result']['point']['y'])
#         park_addr[location]=[x,y]

In [17]:
# for j in park_addr_parcel.values():
#     key = "7F8A26C5-7C0A-318B-A521-FE1AA35942DB"
#     location = j
#     response = requests.get(f"http://api.vworld.kr/req/address?service=address&request=getCoord&key={key}&address={location}&type=PARCEL")
#     if 'result' in response.json()['response']:
#         x= float(response.json()['response']['result']['point']['x'])
#         y = float(response.json()['response']['result']['point']['y'])
#         park_addr[location]=[x,y]

In [18]:
park_addr={'전북 전주시 덕진구 고내천변로 58': [127.103045423, 35.875572562],
 '전북 전주시 덕진구 권삼득로 390 전주덕진공원': [127.121031159, 35.847494653],
 '전북 전주시 덕진구 덕진동1가 640-10': [127.140260164919, 35.8531846654219],
 '전북 전주시 덕진구 덕진동2가 646-1': [127.106251503516, 35.8467190779429],
 '전북 전주시 덕진구 덕진동2가 682': [127.107135229471, 35.8452804957774],
 '전북 전주시 덕진구 덕진동2가 산27-1': [127.119209093, 35.8436809782464],
 '전북 전주시 덕진구 만성동 1353-9': [127.076173219074, 35.8492012628169],
 '전북 전주시 덕진구 서귀로 107 전주시청소년자유센터': [127.087015872, 35.851467962],
 '전북 전주시 덕진구 송천동2가 1356': [127.136496456089, 35.8716157960729],
 '전북 전주시 덕진구 연화길 19': [127.131619788, 35.853371406],
 '전북 전주시 덕진구 완주로 6': [127.147534201, 35.867446974],
 '전북 전주시 덕진구 우아동1가 1020': [127.169120345735, 35.8226859165527],
 '전북 전주시 덕진구 우아동2가 907': [127.171112272315, 35.830531628867],
 '전북 전주시 덕진구 우아동2가 910-4': [127.170499511538, 35.8287199005855],
 '전북 전주시 덕진구 장동': [127.068032858205, 35.8518395334944],
 '전북 전주시 덕진구 중동 830': [127.053611664834, 35.8328441574672],
 '전북 전주시 덕진구 팔복동3가 46-1': [127.10105121931, 35.8394422108349],
 '전북 전주시 완산구 강변로 96-14': [127.110970348, 35.799140044],
 '전북 전주시 완산구 거마남로 41': [127.121451092, 35.794903085],
 '전북 전주시 완산구 거마평로 109-16': [127.115265436, 35.802354133],
 '전북 전주시 완산구 공수내1길 19-4': [127.146284231, 35.80543043],
 '전북 전주시 완산구 메너머4길 11': [127.12158197, 35.824817773],
 '전북 전주시 완산구 문화3길 14-3': [127.15725209, 35.824469227],
 '전북 전주시 완산구 삼천동2가 730-15': [127.111317435705, 35.7912565398038],
 '전북 전주시 완산구 서서학동 296-1': [127.148919799295, 35.8037067991491],
 '전북 전주시 완산구 서서학동 산27-2': [127.145166593557, 35.8018898234845],
 '전북 전주시 완산구 서신동 887-8': [127.119214444323, 35.8261883498716],
 '전북 전주시 완산구 서신동 944-1': [127.108622335014, 35.8360394473194],
 '전북 전주시 완산구 서원로 238': [127.120151128, 35.811048729],
 '전북 전주시 완산구 성지산로 6': [127.12002817, 35.79902943],
 '전북 전주시 완산구 송정로 15': [127.118333426, 35.802418927],
 '전북 전주시 완산구 여울로 77': [127.116085374, 35.837419759],
 '전북 전주시 완산구 우전로 130': [127.099514051, 35.806385085],
 '전북 전주시 완산구 우전로 177': [127.101115561, 35.810696349],
 '전북 전주시 완산구 유연로 217-24': [127.10976908, 35.827667921],
 '전북 전주시 완산구 인정2길 5': [127.11947283, 35.819292247],
 '전북 전주시 완산구 중화산동1가 150-3': [127.137781737641, 35.8141505590001],
 '전북 전주시 완산구 중화산동2가 702-1': [127.118925545189, 35.8224943971189],
 '전북 전주시 완산구 태평동 293': [127.137678799514, 35.8240594634563],
 '전북 전주시 완산구 평화동1가 722-1': [127.133713394539, 35.7952224342245],
 '전북 전주시 완산구 평화동2가 230-133': [127.134669726168, 35.7908462904895],
 '전북 전주시 완산구 평화동2가 858-4': [127.135196730453, 35.7918149635269],
 '전북 전주시 완산구 평화동2가 873-5': [127.13609945516, 35.7889493390376],
 '전북 전주시 완산구 평화동2가 889-3': [127.130410576089, 35.785844841394],
 '전북 전주시 완산구 하거마7길 43': [127.117423246, 35.79814468],
 '전북 전주시 완산구 홍산로 148': [127.104215494, 35.806897163],
 '전북 전주시 완산구 효자동2가 1164-1': [127.108621306877, 35.81300369389],
 '전북 전주시 완산구 효자동2가 1191-3': [127.095643885722, 35.8073799566061],
 '전북 전주시 완산구 효자동2가 1205-1': [127.099096499616, 35.8037570028798],
 '전북 전주시 완산구 효자동2가 1247-13': [127.099182231704, 35.8126571173351],
 '전북 전주시 완산구 효자동2가 1260-18': [127.104229908763, 35.8124386487372],
 '전북 전주시 완산구 효자동2가 1310': [127.095486758524, 35.8093817006575],
 '전북 전주시 완산구 효자동2가 1371': [127.107881176481, 35.8014012540922],
 '전북 전주시 완산구 효자동3가 1526-1': [127.110616739302, 35.8187032952929],
 '전북 전주시 완산구 효자동3가 1604-29': [127.101404379041, 35.8271027797721],
 '전북 전주시 완산구 효자동3가 1615-22': [127.103310484304, 35.8269905768283],
 '전북 전주시 완산구 효자동3가 1641-7': [127.099681435305, 35.8233173843666],
 '전북 전주시 완산구 효자동3가 1687-13': [127.09967843318, 35.8200044195512],
 '전북 전주시 완산구 효자동3가 1713-18': [127.09627981266, 35.816489010774],
 '전북 전주시 완산구 효자동3가 1858': [127.099991493797, 35.8170027052289],
 '전북 전주시 완산구 효자천변1길 31': [127.114512822, 35.800696164]}

park_dict = dict()
for content in arr:
    content_lat = content[0]
    content_long = content[1]
    park_count = 0
    
    for park in park_addr:
        park_loc = park_addr[park]
        if ((content_lat-park_loc[0])**2+(content_long-park_loc[1])**2)**(1/2) <= ((content_lat-(content_lat-0.0055))**2+(content_long-(content_long-0.0055))**2)**(1/2):
            park_count+=1
    park_dict[f'{content[0]}-{content[1]}']=park_count

    

In [19]:
### 강

samchun_river = [
    [35.75293990839722, 127.1218845257207],
    [35.75511660929152, 127.12340802009957],
    [35.76660860253775, 127.11748570302515],
    [35.77155313130564, 127.1179577718571],
    [35.78295571374906, 127.10999697587212],
    [35.78513159342862, 127.10954636467221],
    [35.79265097438983, 127.11222857348669],
    [35.79554017706635, 127.10965365297356],
    [35.80023925799425, 127.10911721121415],
    [35.80248427643983, 127.11169213172728],
    ## 우전초 뒤쪽 천변과 합류함
    [35.80696084528533, 127.10942381769188],
    [35.81035416600382, 127.1115266693626],
    [35.815104571433885, 127.11255663759472],
    [35.824047786857996, 127.11330765611538],
    [35.82585718653191, 127.11229914554667],
    [35.83006736056516, 127.11004609015717],
    [35.8322419498786, 127.10566872515467],
    [35.833790221059935, 127.10538977543976],
    [35.833738032736285, 127.10530394470072],
    [35.836834486500734, 127.10596913248241],
    # 가련교 방향과 합류함
    [35.842905272682664, 127.10463875687375],
    [35.84753197832762, 127.10665577783858],
    [35.8538280387514, 127.11517447299715],
    [35.857636738616016, 127.11536759203054],
    [35.859603224400615, 127.1084601537938],
    [35.86282039692555, 127.10659333650243],
    [35.867863269453544, 127.10708686282348],
    [35.876209387866304, 127.10088559575011],
    [35.87819146174914, 127.09901877842736],
    [35.881190558351186, 127.09236689949877],
    [35.88345504012023, 127.09105798007856],
    [35.88481107724224, 127.09114381076004],
    [35.89061743515467, 127.088461601963],
    [35.89162568163571, 127.08560773137421]
]

samchun_sub_river = [
    [35.801112004526395, 127.09677934944936],
    [35.80123382751581, 127.09718704518652],
    [35.801042391305714, 127.09845304773873],
    [35.8012686340499, 127.10059881477636],
    [35.801059794616606, 127.10162878295444],
    [35.80142526326456, 127.10248708976947],
    [35.80257386806915, 127.10355997328828],
    [35.80266088290788, 127.10435390709222],
    [35.803148164243375, 127.10458994146636],
    [35.804227133705766, 127.10433244942186],
    [35.80497544262878, 127.10486889118127],
    [35.8057063422128, 127.10581302867782],
    [35.80692449323972, 127.10941791730103]
]

dukjin_park_river = [
    [35.85336465473202, 127.115550785082],
    [35.85296464133452, 127.1166451262712],
    [35.852790721836605, 127.11707427967872],
    [35.8528255057667, 127.11795404416415],
    [35.85216460848527, 127.1193487927386],
    [35.85089497456748, 127.12052896460929],
    [35.850338416304126, 127.12185934017265],
    [35.8492774662939, 127.12241723960243]
]

jeonju_river_main = [
    # 색장교 부터 시작
    [35.79823945347732, 127.19003036329433],
    [35.80071077264937, 127.17904403606165],
    [35.80147651791837, 127.17805698322432],
    [35.801546130758624, 127.17595413152745],
    [35.80140690501712, 127.17513874005314],
    [35.80227706189826, 127.1736367031268],
    [35.80168535625612, 127.16784313212517],
    [35.80290356895064, 127.16531112702079],
   [35.81146533675591, 127.16132000033079],
    [35.810456070655114, 127.15011909639432],
    [35.812265780175444, 127.14441135493445],
    [35.81289220843716, 127.13990524415543],
    [35.81400584647803, 127.1389181913181],
    [35.81630267560922, 127.13883236063661],
    [35.81929541394792, 127.13720157754162],
    [35.81919101800061, 127.13733032356386],
    [35.82319276449554, 127.13260963573599],
    [35.832865718242694, 127.1261723342292],
    [35.834605333009925, 127.12398365185082],
    [35.837806124763, 127.1179755025072],
    [35.83804965796133, 127.11213901616482],
    [35.84222439652876, 127.10505798494064],
]

jeonju_river_sub = [
    [35.83303968167341, 127.13394000926722],
    [35.83338760691532, 127.13312461779289],
    [35.83457054132602, 127.12419822691632],
]

ajung_water = [
    [35.8250456052072, 127.17615957474659],
    [35.82563713679587, 127.17688913553941],
    [35.833361438714206, 127.1734559082792],
    [35.83677102647198, 127.16852064409264],
    [35.841328511872305, 127.16577406228446],
    [35.84369412625177, 127.16663236909949],
    [35.84383327784163, 127.16719026852928],
    [35.844285518822694, 127.16731901455155],
    [35.845850948460395, 127.1680914906851],
    [35.84665104500976, 127.16791982932209],
    [35.84741634719958, 127.16693277648477],
    [35.84845992940945, 127.16697569174883],
    [35.853990684696235, 127.16676111504506],
    [35.8554863572744, 127.1681773212899],
    [35.858581962315974, 127.1677910833486],
    [35.861851346313934, 127.16581697710235],
    [35.86532926624582, 127.16328497199795],
    [35.866859502666316, 127.16268415722742],
    [35.87291060267905, 127.15839262315215],
    [35.87819624299659, 127.15358610485539]
]
river_list = [samchun_river, samchun_sub_river, jeonju_river_main, jeonju_river_sub, ajung_water]
river_dict = dict()
for content in arr:
    content_lat = content[0]
    content_long = content[1]
    river_count = 0
    i=1
    
    for river in river_list:
        for river_loc in river:
            if ((content_lat-river_loc[1])**2+(content_long-river_loc[0])**2)**(1/2) <= ((content_lat-(content_lat-0.011))**2+(content_long-(content_long-0.011))**2)**(1/2):
                river_count+=1
                print(f'{content[0]}-{content[1]}:{i}')
                break
        i+=1
    river_dict[f'{content[0]}-{content[1]}']=river_count

# river_map = folium.Map(location=[35.83001,127.125001], zoom_start=13)

# for water in [samchun_river, samchun_sub_river, jeonju_river_main, jeonju_river_sub, ajung_water]:    
#     for point in water:
#         lat = point[1]
#         long = point[0]
#         folium.Marker([long, lat], popup=f'{long}-{lat}의 원룸', icon=folium.Icon(color='blue',icon='info-sign')).add_to(river_map)
#         folium.Circle([long, lat], radius=500, color='#bdf9f8', fill_color='#bdf9f8', popup=f'{long}-{lat}의 강변').add_to(river_map)
# #     folium.Circle([long, lat], radius=500, color='#ebe134', fill_color='#ebe134', popup=popup).add_to(cctv_map)
# river_map.save('river_point.html')

127.08-35.88:1
127.14-35.84:4
127.13-35.83:3
127.13-35.83:4
127.13-35.84:3
127.13-35.84:4
127.12-35.84:1
127.12-35.84:3
127.12-35.84:4
127.12-35.86:1
127.16-35.85:5
127.11-35.86:1
127.1-35.85:1
127.1-35.85:3
127.11-35.85:1
127.11-35.85:3
127.14-35.81:3
127.14-35.82:3
127.14-35.82:4
127.12-35.8:1
127.12-35.8:2
127.15-35.8:3
127.12-35.83:1
127.12-35.83:3
127.12-35.83:4
127.13-35.81:3
127.16-35.82:3
127.14-35.8:3
127.12-35.81:1
127.12-35.81:2
127.11-35.82:1
127.11-35.82:2
127.14-35.83:3
127.14-35.83:4
127.13-35.85:1
127.12-35.85:1
127.12-35.85:3
127.11-35.84:1
127.11-35.84:3
127.11-35.84:4
127.17-35.84:5
127.17-35.83:5
127.13-35.86:1
127.14-35.88:5
127.12-35.87:1
127.11-35.87:1
127.17-35.82:3
127.17-35.82:5
127.18-35.83:5
127.16-35.84:5
127.15-35.86:5
127.16-35.86:5
127.16-35.83:5
127.11-35.88:1
127.1-35.89:1
127.13-35.82:3
127.13-35.82:4
127.1-35.86:1
127.1-35.84:1
127.1-35.84:3
127.16-35.87:5
127.15-35.89:5
127.15-35.82:3
127.15-35.81:3
127.16-35.81:3
127.18-35.79:3
127.17-35.8:3
127.18

In [20]:
#### 보증금 / 월세
deokjin_file_list = [
    './exchange/deokjin/단독다가구(전월세)_실거래가_20210428221912_덕진.xlsx',
    './exchange/deokjin/연립다세대(전월세)_실거래가_20210428221838_덕진.xlsx',
    './exchange/deokjin/오피스텔(전월세)_실거래가_20210428_덕진.xlsx'
]

wansan_file_list = [
    './exchange/wansan/단독다가구(전월세)_실거래가_20210428222031_완산.xlsx',
    './exchange/wansan/연립다세대(전월세)_실거래가_20210428222005_완산.xlsx',
    './exchange/wansan/오피스텔(전월세)_실거래가_20210428_완산.xlsx'
]



deokjin_trade_mono = pd.read_excel(deokjin_file_list[0], header=16)
deokjin_trade_row = pd.read_excel(deokjin_file_list[1], header=16)
deokjin_trade_office = pd.read_excel(deokjin_file_list[2], header=16)
wansan_trade_mono = pd.read_excel(wansan_file_list[0], header=16)
wansan_trade_row = pd.read_excel(wansan_file_list[1], header=16)
wansan_trade_office = pd.read_excel(wansan_file_list[2], header=16)
deokjin_trade_mono

,시군구,번지,도로조건,계약면적(㎡),전월세구분,계약년월,계약일,보증금(만원),월세(만원),건축년도,도로명
0,전라북도 전주덕진구 고랑동,1***,12m미만,33.00,월세,202010,6,300,25,2014.0,쪽구름5길
1,전라북도 전주덕진구 고랑동,8**,25m미만,45.00,전세,202011,6,"7,000",0,2013.0,여산로
2,전라북도 전주덕진구 고랑동,8**,25m미만,45.00,전세,202012,23,5000,0,2013.0,여산로
3,전라북도 전주덕진구 고랑동,8**,25m미만,45.00,전세,202011,23,"5,000",0,2013.0,여산로
4,전라북도 전주덕진구 고랑동,8**,25m미만,45.00,전세,202011,27,"5,000",0,2013.0,여산로
...,...,...,...,...,...,...,...,...,...,...,...
3679,전라북도 전주덕진구 호성동1가,7**,12m미만,56.34,전세,202009,22,"6,000",0,NaN,호성2길
3680,전라북도 전주덕진구 호성동1가,5**,8m미만,57.38,전세,202102,15,"1,800",0,2013.0,호성3길
3681,전라북도 전주덕진구 호성동1가,5**,8m미만,57.38,전세,202010,24,"5,000",0,2013.0,호성3길
3682,전라북도 전주덕진구 호성동1가,7**,12m미만,80.95,전세,202012,7,"8,500",0,NaN,호성2길


In [21]:
# j = deokjin_trade_mono
# j['보증금(만원)']=j['보증금(만원)'].str.strip()
# j['보증금(만원)']=j['보증금(만원)'].str.replace(',','')
# j.dropna()
# j.drop(j[j['보증금(만원)']=='NaN'].index, inplace=True)
# j['보증금(만원)'] = pd.to_numeric(j['보증금(만원)'], errors='coerce')

# j['address']=j['시군구']+' '+j['도로명']
# j['dong']=j['시군구'].map(lambda x: x.split()[2])
# j_count = j.groupby(['전월세구분','dong','도로명']).count()
# j_mean = j.groupby(['전월세구분','dong','도로명']).mean()

# j_count.to_csv('deokjin_mono_count.csv',sep=',',na_rep='NaN', encoding='cp949', index=False)
# j_mean.to_csv('deokjin_mono_mean.csv',sep=',',na_rep='NaN', encoding='cp949', index=False)

# j = deokjin_trade_row
# j['보증금(만원)']=j['보증금(만원)'].str.strip()
# j['보증금(만원)']=j['보증금(만원)'].str.replace(',','')
# j.dropna()
# j.drop(j[j['보증금(만원)']=='NaN'].index, inplace=True)
# j['보증금(만원)'] = pd.to_numeric(j['보증금(만원)'], errors='coerce')

# j['address']=j['시군구']+' '+j['도로명']
# j['dong']=j['시군구'].map(lambda x: x.split()[2])
# j_count = j.groupby(['전월세구분','dong','도로명']).count()
# j_mean = j.groupby(['전월세구분','dong','도로명']).mean()

# j_count.to_csv('deokjin_row_count.csv',sep=',',na_rep='NaN', encoding='cp949', index=False)
# j_mean.to_csv('deokjin_row_mean.csv',sep=',',na_rep='NaN', encoding='cp949', index=False)

# j = deokjin_trade_office
# j['보증금(만원)']=j['보증금(만원)'].str.strip()
# j['보증금(만원)']=j['보증금(만원)'].str.replace(',','')
# j.dropna()
# j.drop(j[j['보증금(만원)']=='NaN'].index, inplace=True)
# j['보증금(만원)'] = pd.to_numeric(j['보증금(만원)'], errors='coerce')

# j['address']=j['시군구']+' '+j['도로명']
# j['dong']=j['시군구'].map(lambda x: x.split()[2])
# j_count = j.groupby(['전월세구분','dong','도로명']).count()
# j_mean = j.groupby(['전월세구분','dong','도로명']).mean()

# j_count.to_csv('deokjin_office_count.csv',sep=',',na_rep='NaN', encoding='cp949', index=False)
# j_mean.to_csv('deokjin_office_mean.csv',sep=',',na_rep='NaN', encoding='cp949', index=False)

In [22]:
# j = wansan_trade_mono
# j['보증금(만원)']=j['보증금(만원)'].str.strip()
# j['보증금(만원)']=j['보증금(만원)'].str.replace(',','')
# j.dropna()
# j.drop(j[j['보증금(만원)']=='NaN'].index, inplace=True)
# j['보증금(만원)'] = pd.to_numeric(j['보증금(만원)'], errors='coerce')

# j['address']=j['시군구']+' '+j['도로명']
# j['dong']=j['시군구'].map(lambda x: x.split()[2])
# j_count = j.groupby(['전월세구분','dong','도로명']).count()
# j_mean = j.groupby(['전월세구분','dong','도로명']).mean()

# j_count.to_csv('wansan_mono_count.csv',sep=',',na_rep='NaN', encoding='cp949', index=False)
# j_mean.to_csv('wansan_mono_mean.csv',sep=',',na_rep='NaN', encoding='cp949', index=False)

# j = wansan_trade_row
# j['보증금(만원)']=j['보증금(만원)'].str.strip()
# j['보증금(만원)']=j['보증금(만원)'].str.replace(',','')
# j.dropna()
# j.drop(j[j['보증금(만원)']=='NaN'].index, inplace=True)
# j['보증금(만원)'] = pd.to_numeric(j['보증금(만원)'], errors='coerce')

# j['address']=j['시군구']+' '+j['도로명']
# j['dong']=j['시군구'].map(lambda x: x.split()[2])
# j_count = j.groupby(['전월세구분','dong','도로명']).count()
# j_mean = j.groupby(['전월세구분','dong','도로명']).mean()

# j_count.to_csv('wansan_row_count.csv',sep=',',na_rep='NaN', encoding='cp949', index=False)
# j_mean.to_csv('wansan_row_mean.csv',sep=',',na_rep='NaN', encoding='cp949', index=False)

# j = wansan_trade_office
# j['보증금(만원)']=j['보증금(만원)'].str.strip()
# j['보증금(만원)']=j['보증금(만원)'].str.replace(',','')
# j.dropna()
# j.drop(j[j['보증금(만원)']=='NaN'].index, inplace=True)
# j['보증금(만원)'] = pd.to_numeric(j['보증금(만원)'], errors='coerce')

# j['address']=j['시군구']+' '+j['도로명']
# j['dong']=j['시군구'].map(lambda x: x.split()[2])
# j_count = j.groupby(['전월세구분','dong','도로명']).count()
# j_mean = j.groupby(['전월세구분','dong','도로명']).mean()

# j_count.to_csv('wansan_office_count.csv',sep=',',na_rep='NaN', encoding='cp949', index=False)
# j_mean.to_csv('wansan_office_mean.csv',sep=',',na_rep='NaN', encoding='cp949', index=False)

In [23]:
# key = "7F8A26C5-7C0A-318B-A521-FE1AA35942DB"

# dj_mono_df = pd.read_csv('deokjin_mono_mean.csv', encoding='cp949')
# dj_mono_df['loc'] = '전북 전주시 '+dj_mono_df['dong']+' '+dj_mono_df['도로명']
# dj_mono_df['x'] = ''
# dj_mono_df['y'] = ''
# locations = list(dj_mono_df['loc'])
# i=0
# dj_mono_df_copy = dj_mono_df.copy()
# for location in locations:
#     response = requests.get(f"http://api.vworld.kr/req/address?service=address&request=getCoord&key={key}&address={location}&type=ROAD")
#     if 'result' in response.json()['response']:
#         dj_mono_df_copy['x'][i] = float(response.json()['response']['result']['point']['x'])
#         dj_mono_df_copy['y'][i] = float(response.json()['response']['result']['point']['y'])
#         i+=1
# dj_mono_df.to_csv('deokjin_mono_mean.csv', encoding='cp949', index=False)

# dj_row_df = pd.read_csv('deokjin_row_mean.csv', encoding='cp949')
# dj_row_df['loc'] = '전북 전주시 '+dj_row_df['dong']+' '+dj_row_df['도로명']
# dj_row_df['x'] = ''
# dj_row_df['y'] = ''
# locations = list(dj_row_df['loc'])
# i=0
# for location in locations:
#     response = requests.get(f"http://api.vworld.kr/req/address?service=address&request=getCoord&key={key}&address={location}&type=ROAD")
#     if 'result' in response.json()['response']:
#         dj_row_df['x'][i] = float(response.json()['response']['result']['point']['x'])
#         dj_row_df['y'][i] = float(response.json()['response']['result']['point']['y'])
#         i+=1
# dj_row_df.to_csv('deokjin_row_mean.csv', encoding='cp949', index=False)

# dj_office_df = pd.read_csv('deokjin_office_mean.csv', encoding='cp949')
# dj_office_df['loc'] = '전북 전주시 '+dj_office_df['dong']+' '+dj_office_df['도로명']
# dj_office_df['x'] = ''
# dj_office_df['y'] = ''
# locations = list(dj_office_df['loc'])
# i=0
# for location in locations:
#     response = requests.get(f"http://api.vworld.kr/req/address?service=address&request=getCoord&key={key}&address={location}&type=ROAD")
#     if 'result' in response.json()['response']:
#         dj_office_df['x'][i] = float(response.json()['response']['result']['point']['x'])
#         dj_office_df['y'][i] = float(response.json()['response']['result']['point']['y'])
#         i+=1
# dj_office_df.to_csv('deokjin_office_mean.csv', encoding='cp949', index=False)

In [24]:
# ws_mono_df = pd.read_csv('wansan_mono_mean.csv', encoding='cp949')
# ws_mono_df['loc'] = '전북 전주시 '+ws_mono_df['dong']+' '+ws_mono_df['도로명']
# ws_mono_df['x'] = ''
# ws_mono_df['y'] = ''
# locations = list(ws_mono_df['loc'])
# i=0
# ws_mono_df_copy = ws_mono_df.copy()
# for location in locations:
#     response = requests.get(f"http://api.vworld.kr/req/address?service=address&request=getCoord&key={key}&address={location}&type=ROAD")
#     if 'result' in response.json()['response']:
#         ws_mono_df_copy['x'][i] = float(response.json()['response']['result']['point']['x'])
#         ws_mono_df_copy['y'][i] = float(response.json()['response']['result']['point']['y'])
#         i+=1
# ws_mono_df_copy.to_csv('wansan_mono_mean.csv', encoding='cp949', index=False)

# ws_row_df = pd.read_csv('wansan_row_mean.csv', encoding='cp949')
# ws_row_df['loc'] = '전북 전주시 '+ws_row_df['dong']+' '+ws_row_df['도로명']
# ws_row_df['x'] = ''
# ws_row_df['y'] = ''
# locations = list(ws_row_df['loc'])
# i=0
# ws_row_df_copy = ws_row_df.copy()
# for location in locations:
#     response = requests.get(f"http://api.vworld.kr/req/address?service=address&request=getCoord&key={key}&address={location}&type=ROAD")
#     if 'result' in response.json()['response']:
#         ws_row_df_copy['x'][i] = float(response.json()['response']['result']['point']['x'])
#         ws_row_df_copy['y'][i] = float(response.json()['response']['result']['point']['y'])
#         i+=1
# ws_row_df_copy.to_csv('wansan_row_mean.csv', encoding='cp949', index=False)

# ws_office_df = pd.read_csv('wansan_office_mean.csv', encoding='cp949')
# ws_office_df['loc'] = '전북 전주시 '+ws_office_df['dong']+' '+ws_office_df['도로명']
# ws_office_df['x'] = ''
# ws_office_df['y'] = ''
# locations = list(ws_office_df['loc'])
# i=0
# ws_office_df_copy = ws_office_df.copy()
# for location in locations:
#     response = requests.get(f"http://api.vworld.kr/req/address?service=address&request=getCoord&key={key}&address={location}&type=ROAD")
#     if 'result' in response.json()['response']:
#         ws_office_df_copy['x'][i] = float(response.json()['response']['result']['point']['x'])
#         ws_office_df_copy['y'][i] = float(response.json()['response']['result']['point']['y'])
#         i+=1
# ws_office_df_copy.to_csv('wansan_office_mean.csv', encoding='cp949', index=False)

In [25]:
dj_mono_df = pd.read_csv('deokjin_mono_mean.csv', encoding='cp949')
dj_mono_amount = pd.read_csv('deokjin_mono_count.csv', encoding='cp949')
dj_row_df = pd.read_csv('deokjin_row_mean.csv', encoding='cp949')
dj_row_amount = pd.read_csv('deokjin_row_count.csv', encoding='cp949')
dj_office_df = pd.read_csv('deokjin_office_mean.csv', encoding='cp949')
dj_office_amount = pd.read_csv('deokjin_office_count.csv', encoding='cp949')

ws_mono_df = pd.read_csv('wansan_mono_mean.csv', encoding='cp949')
ws_mono_amount = pd.read_csv('wansan_mono_count.csv', encoding='cp949')
ws_row_df = pd.read_csv('wansan_row_mean.csv', encoding='cp949')
ws_row_amount = pd.read_csv('wansan_row_count.csv', encoding='cp949')
ws_office_df = pd.read_csv('wansan_office_mean.csv', encoding='cp949')
ws_office_amount = pd.read_csv('wansan_office_count.csv', encoding='cp949')

dj_mono_df = pd.concat([dj_mono_df, dj_mono_amount['번지']], axis=1)
dj_mono_df.to_csv('deokjin_mono_mean.csv', encoding='cp949', index=False)
dj_row_df = pd.concat([dj_row_df, dj_row_amount['번지']], axis=1)
dj_row_df.to_csv('deokjin_row_mean.csv', encoding='cp949', index=False)
dj_office_df = pd.concat([dj_office_df, dj_office_amount['번지']], axis=1)
dj_office_df.to_csv('deokjin_office_mean.csv', encoding='cp949', index=False)

ws_mono_df = pd.concat([ws_mono_df, ws_mono_amount['번지']], axis=1)
ws_mono_df.to_csv('wansan_mono_mean.csv', encoding='cp949', index=False)
ws_row_df = pd.concat([ws_row_df, ws_row_amount['번지']], axis=1)
ws_row_df.to_csv('wansan_row_mean.csv', encoding='cp949', index=False)
ws_office_df = pd.concat([ws_office_df, ws_office_amount['번지']], axis=1)
ws_office_df.to_csv('wansan_office_mean.csv', encoding='cp949', index=False)

In [63]:
estate_map = folium.Map(location=[35.83001,127.125001], zoom_start=13)
dj_mono_df = pd.read_csv('deokjin_mono_mean.csv', encoding='cp949')
dj_row_df = pd.read_csv('deokjin_row_mean.csv', encoding='cp949')
dj_office_df = pd.read_csv('deokjin_office_mean.csv', encoding='cp949')

ws_mono_df = pd.read_csv('wansan_mono_mean.csv', encoding='cp949')
ws_row_df = pd.read_csv('wansan_row_mean.csv', encoding='cp949')
ws_office_df = pd.read_csv('wansan_office_mean.csv', encoding='cp949')

dj_mono_df.dropna(inplace=True)
dj_row_df.dropna(inplace=True)
dj_office_df.dropna(inplace=True)
ws_mono_df.dropna(inplace=True)
ws_row_df.dropna(inplace=True)
ws_office_df.dropna(inplace=True)
estate_arr = []

mono_dict = {}
row_dict = {}
office_dict = {}
for content in arr_dict:
    lat,long = map(float, content.split('-'))
    long_round = round(long, 2)
    lat_round = round(lat, 2)
    mono_dict.setdefault(f'{lat_round}-{long_round}', [])
    row_dict.setdefault(f'{lat_round}-{long_round}', [])
    office_dict.setdefault(f'{lat_round}-{long_round}', [])
    
def placing_marker(df_arr, color, div):
    for i in df_arr:
        doc = i
        doc_x = list(map(float, list(doc['x'])))
        doc_y = list(map(float, list(doc['y'])))
        doc_contract_type = list(doc['전월세구분'])
        doc_deposit = list(doc['보증금(만원)'])
        doc_fee = list(doc['월세(만원)'])
        doc_dong = list(doc['dong'])
        doc_road = list(doc['도로명'])
        doc_count = list(doc['번지'])
        for j in range(len(doc_x)):
            lat = float(doc_x[j])
            long = float(doc_y[j])
            c_type = doc_contract_type[j]
            deposit = doc_deposit[j]
            fee = doc_fee[j]
            loc = doc_dong[j]+' '+doc_road[j]
            count = doc_count[j]
            iframe = folium.IFrame(f'''
                        <b>{loc} 지역</b>의<br>
                        최근 1년간 {div} 거래종목의<br>
                        계약유형 : {c_type}<br>
                        계약건수 : {count}<br>
                        평균 월세 : {fee:.2f}만원 <br>
                        보증금 : {deposit:.2f}만원
                            ''')
            popup = folium.Popup(iframe,
                     min_width=400,
                     max_width=200)
            
    
            long_round = round(long, 2)
            lat_round = round(lat, 2)
            location = str(lat_round)+'-'+str(long_round)
            if div=='단독주택':
                if f'{lat_round}-{long_round}' not in mono_dict.keys():
                    mono_dict[f'{lat_round}-{long_round}']=[[deposit, fee, count, c_type]]
                else:
                    mono_dict[f'{lat_round}-{long_round}'].append([deposit, fee, count, c_type])
            elif div=='연립주택':
                if f'{lat_round}-{long_round}' not in row_dict.keys():
                    row_dict[f'{lat_round}-{long_round}']=[[deposit, fee, count, c_type]]
                else:
                    row_dict[f'{lat_round}-{long_round}'].append([deposit, fee, count, c_type])
            else:
                if f'{lat_round}-{long_round}' not in office_dict.keys():
                    office_dict[f'{lat_round}-{long_round}']=[[deposit, fee, count, c_type]]
                else:
                    office_dict[f'{lat_round}-{long_round}'].append([deposit, fee, count, c_type])
            
            if [long, lat] not in estate_arr:
                folium.Marker([long, lat], popup=popup, icon=folium.Icon(color=color,icon='info-sign')).add_to(estate_map)
                estate_arr.append([long,lat])
            else:
                if [long, lat+0.0003] not in estate_arr:
                    folium.Marker([long, lat+0.0003], popup=popup, icon=folium.Icon(color=color,icon='flag')).add_to(estate_map)
                    estate_arr.append([long, lat+0.0003])
                else:
                    if [long, lat+0.0003] not in estate_arr:
                        folium.Marker([long, lat+0.0006], popup=popup, icon=folium.Icon(color=color,icon='home')).add_to(estate_map)
                        estate_arr.append([long, lat+0.0006])
                    else:
                        if [long, lat+0.0006] not in estate_arr:
                            folium.Marker([long, lat+0.0009], popup=popup, icon=folium.Icon(color=color,icon='star')).add_to(estate_map)
                            estate_arr.append([long, lat+0.0009])
                        else:
                            if [long, lat+0.0009] not in estate_arr:
                                print(f"{long}-{lat+0.0012}-{loc}-{c_type}")
                                folium.Marker([long, lat+0.0012], popup=popup, icon=folium.Icon(color=color,icon='star')).add_to(estate_map)
                                estate_arr.append([long, lat+0.0012])

                
placing_marker([dj_mono_df, ws_mono_df], 'red', '단독주택')
placing_marker([dj_row_df, ws_row_df], 'blue', '연립주택')
placing_marker([dj_office_df, ws_office_df], 'green', '오피스텔')

estate_map.save('estate_point.html')
print("MONO")
pprint.pprint(mono_dict)
print("-----------")
print("ROW")
pprint.pprint(row_dict)
print("------------")
print("OFFICE")
pprint.pprint(office_dict)

MONO
{'127.04-35.88': [],
 '127.06-35.83': [],
 '127.06-35.84': [[15000.0, 37.5, 2, '월세'],
                  [3363.636364, 44.81818182, 11, '월세'],
                  [3575.0, 37.75, 4, '월세'],
                  [9000.0, 0.0, 3, '전세'],
                  [8000.0, 0.0, 1, '전세']],
 '127.06-35.85': [[2000.0, 48.0, 1, '월세']],
 '127.06-35.88': [],
 '127.07-35.83': [[1200.0, 32.78571429, 14, '월세'],
                  [9666.666667, 41.66666667, 3, '월세'],
                  [19000.0, 10.0, 1, '월세'],
                  [2428.571429, 42.85714286, 7, '월세'],
                  [10000.0, 35.0, 1, '월세'],
                  [5555.555556, 0.0, 27, '전세'],
                  [7000.0, 0.0, 1, '전세'],
                  [15125.0, 0.0, 8, '전세'],
                  [23000.0, 0.0, 3, '전세'],
                  [18000.0, 0.0, 2, '전세'],
                  [15000.0, 0.0, 2, '전세'],
                  [17500.0, 0.0, 2, '전세']],
 '127.07-35.84': [[2800.0, 54.16666667, 6, '월세'],
                  [6500.0, 38.5, 2, '월세'],
           

                  [2500.0, 0.0, 1, '전세']],
 '127.11-35.82': [[300.0, 30.0, 1, '월세'], [6500.0, 0.0, 2, '전세']],
 '127.11-35.83': [],
 '127.11-35.84': [],
 '127.11-35.85': [[500.0, 30.0, 1, '월세'],
                  [3000.0, 0.0, 2, '전세'],
                  [3500.0, 0.0, 2, '전세'],
                  [2000.0, 0.0, 1, '전세'],
                  [2000.0, 0.0, 1, '전세']],
 '127.11-35.86': [],
 '127.11-35.87': [],
 '127.11-35.88': [],
 '127.12-35.79': [[10500.0, 0.0, 1, '전세']],
 '127.12-35.8': [[1000.0, 65.0, 1, '월세'], [18000.0, 0.0, 1, '전세']],
 '127.12-35.81': [[300.0, 35.0, 1, '월세'],
                  [2000.0, 25.0, 1, '월세'],
                  [3000.0, 10.0, 1, '월세'],
                  [6000.0, 20.0, 1, '월세'],
                  [6000.0, 0.0, 1, '전세'],
                  [5000.0, 0.0, 2, '전세'],
                  [7700.0, 0.0, 2, '전세']],
 '127.12-35.82': [[2000.0, 10.0, 1, '월세'],
                  [500.0, 280.0, 1, '월세'],
                  [1000.0, 50.0, 1, '월세'],
                  [100.0, 30.0, 1, 

In [27]:
bus_data = pd.read_csv('bus_df.csv', encoding='cp949')

bus_dict = {}

bus_x = list(bus_data['0'])
bus_y = list(bus_data['1'])
bus_near = bus_data['near_bus_stop']
total_bus = list(bus_data['total_bus'])
conv_dict.setdefault(f'{content[0]}-{content[1]}', 0)


for i in range(len(bus_x)):
#     bus_near_arr = []
#     for j in bus_near:
#         print(j)
#         bus_near_arr.append(j)
    bus_dict.setdefault(f'{bus_x[i]}-{bus_y[i]}',  total_bus[i])


# 종합

지금껏 수집한 데이터들을 종합적으로 표시한다.

In [85]:
total_map = folium.Map(location=[35.83001,127.125001], zoom_start=13)

for content in arr_dict:
    lat,long = map(float, content.split('-'))
    house_amount = arr_dict[f'{lat}-{long}']['value']
    addr = max(arr_dict[f'{lat}-{long}']['addr'])
    park = park_dict[f'{lat}-{long}']
    river = river_dict[f'{lat}-{long}']
    cctv = cctv_dict[f'{lat}-{long}']
    pol = pol_dict[f'{lat}-{long}']
    conv = conv_dict[f'{lat}-{long}']
    bus = bus_dict[f'{lat}-{long}']
    mono_total_deposit_month=1
    mono_total_fee_month=1
    mono_total_count_month=1
    mono_total_deposit_all=1
    mono_total_fee_all=1
    mono_total_count_all=1
    row_total_deposit_month=1
    row_total_fee_month=1
    row_total_count_month=1
    row_total_deposit_all=1
    row_total_fee_all=1
    row_total_count_all=1
    office_total_deposit_month=1
    office_total_fee_month=1
    office_total_count_month=1
    office_total_deposit_all=1
    office_total_fee_all=1
    office_total_count_all=1

    if len(mono_dict[f'{lat}-{long}']) > 1:
        mono = mono_dict[f'{lat}-{long}']
#       deposit, fee, count, c_type
        for item in mono:
            if item[3]=='월세':
                mono_total_deposit_month += item[0]*item[2]
                mono_total_fee_month += item[1]*item[2]
                mono_total_count_month += item[2]
            else:
                mono_total_deposit_all += item[0]*item[2]
                mono_total_fee_all += item[1]*item[2]
                mono_total_count_all += item[2]                
    else :
        mono = '기록 없음'
    if len(row_dict[f'{lat}-{long}']) > 1:
        row = row_dict[f'{lat}-{long}']
        for item in row:
            if item[3]=='월세':
                row_total_deposit_month += item[0]*item[2]
                row_total_fee_month += item[1]*item[2]
                row_total_count_month += item[2]
            else:
                row_total_deposit_all += item[0]*item[2]
                row_total_fee_all += item[1]*item[2]
                row_total_count_all += item[2]    
    else :
        row = '기록 없음'
    if len(office_dict[f'{lat}-{long}']) > 1:
        office = office_dict[f'{lat}-{long}']
        for item in office:
            if item[3]=='월세':
                office_total_deposit_month += item[0]*item[2]
                office_total_fee_month += item[1]*item[2]
                office_total_count_month += item[2]
            else:
                office_total_deposit_all += item[0]*item[2]
                office_total_fee_all += item[1]*item[2]
                office_total_count_all += item[2]    
    else :
        office = '기록 없음'
    
    mono_avg_deposit_month=0
    mono_avg_fee_month=0

    if mono_total_count_month!=1:
        mono_avg_deposit_month = mono_total_deposit_month/(mono_total_count_month-1)
        mono_avg_fee_month = mono_total_fee_month/(mono_total_count_month-1)
        
    mono_avg_deposit_all=0
    if mono_total_count_all!=1:
        mono_avg_deposit_all = mono_total_deposit_all/(mono_total_count_all-1)


    row_avg_deposit_month=0
    row_avg_fee_month=0

    if row_total_count_month!=1:
        row_avg_deposit_month = row_total_deposit_month/(row_total_count_month-1)
        row_avg_fee_month = row_total_fee_month/(row_total_count_month-1)
        
    row_avg_deposit_all=0
    if row_total_count_all!=1:
        row_avg_deposit_all = row_total_deposit_all/(row_total_count_all-1)    
        
    office_avg_deposit_month=0
    office_avg_fee_month=0

    if office_total_count_month!=1:
        office_avg_deposit_month = office_total_deposit_month/(office_total_count_month-1)
        office_avg_fee_month = office_total_fee_month/(office_total_count_month-1)
        
    office_avg_deposit_all=0
    if office_total_count_all!=1:
        office_avg_deposit_all = office_total_deposit_all/(office_total_count_all-1)            
        
    iframe = folium.IFrame(f'''
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.0/dist/css/bootstrap.min.css" rel="stylesheet" integrity="sha384-wEmeIV1mKuiNpC+IOBjI7aAzPcEZeedi5yW5f2yOq55WWLwNGmvvx4Um1vskeMj0" crossorigin="anonymous">
    <script src="https://cdn.jsdelivr.net/npm/bootstrap@5.0.0/dist/js/bootstrap.bundle.min.js" integrity="sha384-p34f1UUtsS3wqzfto5wAAmdvj+osOnFyQFpp4Ua3gs/ZVWx6oOypYoCJhGGScy+8" crossorigin="anonymous"></script>
            <h3>{addr}</h3>
            <table class="table text-center table-hover">
            <tr>
            <td class="table-primary">
            인근주택
            </td>
            <td class="table-primary">
            공원
            </td>
            <td class="table-primary">
            강
            </td>
            <tr>
            <td>
            {house_amount}
            </td>
            <td>
            {park}
            </td>
            <td>
            {river}
            </td>
            <tr>
            <td class="table-primary">
            cctv
            </td>
            <td class="table-primary">
            인근 경찰서
            </td>
            <td class="table-primary">
            인근 편의점
            </td>
            <tr>
            <td>
            {cctv}
            </td>
            <td>
            {pol}
            </td>
            <td>
            {conv}
            </td>
            <tr>
            <td class="table-primary" col-span=3>
            인근 버스 노선
            </td>
            <tr>
            <td >
            {bus}
            </td>
            </table>
            <table class="table text-center table-hover">
            <thead>
            <tr>
            <th scope="col">
            최근 1년간
            </th>
            <th class="table-primary">
            단독주택
            </th>
            <th class="table-success">
            연립주택
            </th>
            <th class="table-info">
            오피스텔
            </th>
            </tr>
            <tr>
            </thead>
            <tbody>
            <td scope="row">
            평균 보증금
            (단위:만원)
            </td>
            <td>
            월세 : {mono_avg_deposit_month:.2f}<br>
            전세 : {mono_avg_deposit_all:.2f}
            </td>
            <td>
            월세 : {row_avg_deposit_month:.2f}<br>
            전세 : {row_avg_deposit_all:.2f}
            </td>
            <td>
            월세 : {office_avg_deposit_month:.2f}<br>
            전세 : {office_avg_deposit_all:.2f}
            </td>
            </tr>
            <tr>
            <td scope="row">
            평균 월세
            (단위:만원)
            </td>
            <td>
            월세 : {mono_avg_fee_month:.2f}
            </td>
            <td>
            월세 : {row_avg_fee_month:.2f}
            </td>
            <td>
            월세 : {office_avg_fee_month:.2f}
            </td>
            </tr>
            <tr>
            <td scope="row">
            평균 거래 수
            (단위:건)
            </td>
            <td>
            월세 : {mono_total_count_month-1}<br>
            전세 : {mono_total_count_month-1}
            </td>
            <td>
            월세 : {row_total_count_month-1}<br>
            전세 : {row_total_count_month-1}
            </td>
            <td>
            월세 : {office_total_count_month-1}<br>
            전세 : {office_total_count_month-1}
            </td>
            </tr>
            </tbody>
            </table>
                ''')

    area100_iframe = folium.IFrame(f'''
            {addr}의 원룸 반경 100m
                ''')

    area500_iframe = folium.IFrame(f'''
            {addr}의 원룸 반경 500m
                ''')

    popup = folium.Popup(iframe,
             min_width=600,
             max_width=800)

    area100_popup = folium.Popup(area100_iframe,
             min_width=400,
             max_width=600)

    area500_popup = folium.Popup(area500_iframe,
             min_width=400,
             max_width=600)    

    folium.Marker([long, lat], popup=popup, icon=folium.Icon(color='red',icon='info-sign')).add_to(total_map)
#     folium.Circle([long, lat], radius=100, color='#3186cc', fill_color='#3186cc', popup=area100_popup).add_to(total_map)
    folium.Circle([long, lat], radius=500, color='#ebe134', fill_color='#ebe134', popup=area500_popup).add_to(total_map)

(total_map).save('total.html')